# Stock Price Prediction Using LSTM

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
dataset_train=pd.read_csv('train.csv')


In [4]:
dataset_train.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-07-31,588.960022,591.989990,577.679993,579.549988,579.549988,2309500
1,2014-08-01,578.549988,583.429993,570.299988,573.599976,573.599976,2213300
2,2014-08-04,576.510010,583.820007,572.260010,582.270020,582.270020,1519400


In [12]:
training_set=dataset_train.iloc[:,1:2].values

In [13]:
training_set

array([[ 588.960022],
       [ 578.549988],
       [ 576.51001 ],
       ...,
       [1228.      ],
       [1242.5     ],
       [1227.      ]])

### feature scaling
two types
$$ Standardization,z_{std}=\frac{x-mean(x)}{std(x)}$$

$$ Normalization,z_{mean}=\frac{x-min(x)}{max(x)-min(x)}$$



In [32]:
from sklearn.preprocessing import MinMaxScaler

In [33]:
sc= MinMaxScaler(feature_range=(0,1))
training_set_scaled=sc.fit_transform(training_set)

In [34]:
# creating the feature vectors and y-labels
X_train=[]
y_train=[]
for i in range(60,1258):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
    
X_train=np.array(X_train)
y_train=np.array(y_train)


In [ ]:
X_train.shapeape
y_train.shape

In [46]:
# reshaping 
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

## Building the RNN

In [47]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import RNN


In [48]:
regressor= Sequential()

In [49]:
# adding the first LSTM layer; also Dropout
regressor.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(rate=0.2))

In [50]:
# adding 2nd lSTM layer
regressor.add(LSTM(units=50,return_sequences=True))
# add dropout
regressor.add(Dropout(rate=0.2))


In [51]:
# adding 23rd lSTM layer
regressor.add(LSTM(units=50,return_sequences=True))
# add dropout
regressor.add(Dropout(rate=0.2))


In [52]:
regressor.add(LSTM(units=50))
# add dropout
regressor.add(Dropout(rate=0.2))

In [53]:
regressor.add(Dense(units=1))

## Compiling the RNN

In [54]:
regressor.compile(optimizer='adam',loss='mean_squared_error')

## Train the RNN

In [55]:
regressor.fit(X_train,y_train,batch_size=32,epochs=100)

W0908 10:32:13.838573 17908 deprecation.py:323] From C:\Users\KINJAL\.conda\envs\py3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0908 10:32:19.602575 17908 deprecation_wrapper.py:119] From C:\Users\KINJAL\.conda\envs\py3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/100
1198/1198 [==============================] - 33s 28ms/step - loss: 0.0479
Epoch 2/100
1198/1198 [==============================] - 20s 17ms/step - loss: 0.0059
Epoch 3/100
1198/1198 [==============================] - 19s 16ms/step - loss: 0.0055
Epoch 4/100
1198/1198 [==============================] - 18s 15ms/step - loss: 0.0049
Epoch 5/100
1198/1198 [==============================] - 18s 15ms/step - loss: 0.0051
Epoch 6/100
1198/1198 [==============================] - 18s 15ms/step - loss: 0.0052
Epoch 7/100
1198/1198 [==============================] - 18s 15ms/step - loss: 0.0047
Epoch 8/100
1198/1198 [==============================] - 19s 16ms/step - loss: 0.0052
Epoch 9/100
1198/1198 [==============================] - 19s 16ms/step - loss: 0.0050
Epoch 10/100
1198/1198 [==============================] - 19s 16ms/step - loss: 0.0050
Epoch 11/100
1198/1198 [==============================] - 20s 16ms/step - loss: 0.0045
Epoch 12/100
1198/1198 [============================

1198/1198 [==============================] - 44s 37ms/step - loss: 0.0018
Epoch 96/100
1198/1198 [==============================] - 44s 37ms/step - loss: 0.0015
Epoch 97/100
1198/1198 [==============================] - 46s 39ms/step - loss: 0.0015
Epoch 98/100
1198/1198 [==============================] - 36s 30ms/step - loss: 0.0015
Epoch 99/100
1198/1198 [==============================] - 33s 27ms/step - loss: 0.0015
Epoch 100/100
1198/1198 [==============================] - 33s 28ms/step - loss: 0.0015


In [ ]:
# saving your trained model to disk
# serialize model to json
model_json= regressor.to_json()
with open('SavedModel/google-lstm-model.json',"w") as json_file:
    json_file.write(model_json)
    
#serialize weight to hdf5
regressor.save_weights("SavedModel/google-lstm-model.h5")
print("Saved model to disk")


In [ ]:
# loading your pre-trained model from disk
from keras.model import model_from_json

# loading the json & creating model
json_file=open("SavedModel/google-lstm-model.json","r")
loaded_model_json =json_file.read()
json_file.close()
#creating the model
loaded_regressor_model=model_from_json(loaded_model_json)
#load the weightd
loaded_regressor_model.load_weights("SavedModel/google-lstm-model.h5")

## Making Predictions

In [ ]:
dataset_test=pd.read_csv('Test.csv')

In [ ]:
real_stock_price=dataset_test.iloc[:,1:2].values

dataset_total=pd.concat((dataset_train['Open'],dataset_test['Open'],axis=0))
